# GroupBy()
- group dataframe using a mapper or by a series of columns


## 1. splitting: the data into groups based on some criteria
## 2. applying: a function to each group independently

#### aggregation 
- compute a summary statistics for each group
- sum, means, sizes, counts

#### transformation 
- perform some group-specific computations and return a like-indexed object
- standarise data (zscore), fillna within groups with a value derived from each group

#### filteration
- discard some groups, according to a group-wise computation that evaluates to True or False
- discard data that belong to groups with only a few members, filter out data based on the group sum or mean

## 3. combinining: the result into a data structure

In [1]:
import pandas as pd
import numpy as np
pd.__version__ # latest is 2.1.2 Nov/2023

/Users/soyeonkim/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


'2.0.3'

## Splitting an object into groups

In [2]:
speeds = pd.DataFrame(
    [
        ("bird", "Falconiformes", 389.0),
        ("bird", "Psittaciformes", 24.0),
        ("mammal", "Carnivora", 80.2),
        ("mammal", "Primates", np.nan),
        ("mammal", "Carnivora", 58)
    ],
    index=['falcon', 'parrot', 'lion', 'monkey', 'leopard'],
    columns=['class', 'order', 'max_speed']
)
speeds

,class,order,max_speed
falcon,bird,Falconiformes,389.0
parrot,bird,Psittaciformes,24.0
lion,mammal,Carnivora,80.2
monkey,mammal,Primates,NaN
leopard,mammal,Carnivora,58.0


In [3]:
grouped = speeds.groupby('class')

In [4]:
grouped = speeds.groupby(['class', 'order'])

In [5]:
df = pd.DataFrame(
    {
        'A':['foo', 'bar', 'foo', 'bar', 'foo', 'bar', 'foo', 'foo'],
        'B':['one', 'one', 'two', 'three', 'two', 'two', 'one', 'three'],
        'C': np.random.randn(8),
        'D': np.random.randn(8)
    }
)
df

,A,B,C,D
0,foo,one,0.149050,1.185864
1,bar,one,1.085320,-0.207327
2,foo,two,0.459395,-0.671467
3,bar,three,-0.928839,0.243055
4,foo,two,0.633772,-0.323280
5,bar,two,-0.119015,1.679083
6,foo,one,-0.510130,-0.754858
7,foo,three,-0.025975,-0.757988


In [6]:
grouped = df.groupby('A')
grouped = df.groupby(['A', 'B'])

In [7]:
df2 = df.set_index(['A', 'B'])
df2

C         D
A   B                        
foo one    0.149050  1.185864
bar one    1.085320 -0.207327
foo two    0.459395 -0.671467
bar three -0.928839  0.243055
foo two    0.633772 -0.323280
bar two   -0.119015  1.679083
foo one   -0.510130 -0.754858
    three -0.025975 -0.757988

In [8]:
grouped = df2.groupby(level=df2.index.names.difference(['B']))
grouped.sum()

,C,D
A,,
bar,0.037467,1.714811
foo,0.706112,-1.321728


In [9]:
def get_letter_type(letter):
    if letter.lower() in 'aeiou':
        return 'vowel'
    else:
        return 'consonant'
    
grouped = df.T.groupby(get_letter_type)    

In [10]:
lst = [1, 2, 3, 1, 2, 3]
s = pd.Series([1, 2, 3, 10, 20, 30], lst)
s

1     1
2     2
3     3
1    10
2    20
3    30
dtype: int64

In [11]:
grouped = s.groupby(level=0)
grouped.first()

1    1
2    2
3    3
dtype: int64

In [12]:
grouped.last()

1    10
2    20
3    30
dtype: int64

In [13]:
grouped.sum()

1    11
2    22
3    33
dtype: int64

## GroupBy sorting

In [14]:
df2 = pd.DataFrame(
    {'X':['B', 'B', 'A', 'A'], 'Y':[1, 2, 3, 4]}
)
df2.groupby(['X']).sum()

,Y
X,
A,7
B,3


In [15]:
df2.groupby(['X'], sort=False).sum()

,Y
X,
B,3
A,7


In [16]:
df3 = pd.DataFrame(
    {'X':['A', 'B', 'A', 'B'], 'Y':[1, 4, 3, 2]}
)
df3.groupby(['X']).get_group('A')

,X,Y
0,A,1
2,A,3


In [17]:
df3.groupby(['X']).get_group('B')

,X,Y
1,B,4
3,B,2


### GroupBy dropna

In [18]:
df_list = [[1, 2, 3], [1, None, 4], [2, 1, 3], [1, 2, 2]]
df_dropna = pd.DataFrame(df_list, columns=['a', 'b', 'c'])
df_dropna

,a,b,c
0,1,2.0,3
1,1,NaN,4
2,2,1.0,3
3,1,2.0,2


In [19]:
df_dropna.groupby(by=['b']).sum()

,a,c
b,,
1.0,2,3
2.0,2,5


In [20]:
df_dropna.groupby(by=['b'], dropna=False).sum()

,a,c
b,,
1.0,2,3
2.0,2,5
NaN,1,4


## GroupBy object attributes

In [23]:
df.groupby('A').groups

{'bar': [1, 3, 5], 'foo': [0, 2, 4, 6, 7]}

In [25]:
df.T.groupby(get_letter_type).groups

{'consonant': ['B', 'C', 'D'], 'vowel': ['A']}

In [28]:
grouped = df.groupby(['A', 'B'])
grouped.groups

{('bar', 'one'): [1], ('bar', 'three'): [3], ('bar', 'two'): [5], ('foo', 'one'): [0, 6], ('foo', 'three'): [7], ('foo', 'two'): [2, 4]}

In [29]:
len(grouped)

6

In [31]:
n = 10
weight = np.random.normal(166, 20, size=n)
height = np.random.normal(60, 10, size=n)
time = pd.date_range('1/1/2023', periods=n)
gender = np.random.choice(['male', 'female'], size=n)

df = pd.DataFrame(
    {"height":height, "weight":weight, "gender":gender}, index=time
)
df

,height,weight,gender
2023-01-01,48.103096,174.619992,male
2023-01-02,56.710490,125.107753,male
2023-01-03,74.990905,172.136251,male
2023-01-04,78.048035,151.224611,male
2023-01-05,56.018274,173.647435,male
2023-01-06,57.754582,175.954640,male
2023-01-07,47.520853,154.593374,male
2023-01-08,71.609585,165.000617,female
2023-01-09,55.863731,172.968026,female
2023-01-10,63.337473,186.425347,male


In [32]:
gb = df.groupby('gender')
gb.groups

{'female': [2023-01-08 00:00:00, 2023-01-09 00:00:00], 'male': [2023-01-01 00:00:00, 2023-01-02 00:00:00, 2023-01-03 00:00:00, 2023-01-04 00:00:00, 2023-01-05 00:00:00, 2023-01-06 00:00:00, 2023-01-07 00:00:00, 2023-01-10 00:00:00]}

## GroupBy with MultiIndex

In [36]:
arrays = [
    ['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'], 
    ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two'],
]
index = pd.MultiIndex.from_arrays(arrays, names=['first', 'second'])
s = pd.DataFrame(np.random.randn(8), index=index)
s

0
first second          
bar   one     2.331489
      two    -0.851381
baz   one     0.646990
      two    -0.318075
foo   one     1.095114
      two     0.061705
qux   one     0.078822
      two     0.823523

In [38]:
grouped = s.groupby(level=0)
grouped.sum()

,0
first,
bar,1.480108
baz,0.328916
foo,1.156820
qux,0.902345


In [39]:
s.groupby(level='second').sum()

,0
second,
one,4.152416
two,-0.284227


In [40]:
arrays = [
    ['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'], 
    ["doo", "doo", "bee", "bee", "bop", "bop", "bop", "bop"],
    ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two'],
]
index = pd.MultiIndex.from_arrays(arrays, names=['first', 'second', 'third'])
s = pd.DataFrame(np.random.randn(8), index=index)
s

0
first second third          
bar   doo    one   -0.852638
             two    0.840500
baz   bee    one   -1.051411
             two   -0.343433
foo   bop    one   -0.163112
             two    2.401102
qux   bop    one    0.875277
             two    0.369104

In [42]:
s.groupby(level=['first', 'second']).sum()

,,0
first,second,
bar,doo,-0.012138
baz,bee,-1.394844
foo,bop,2.237990
qux,bop,1.244382


In [43]:
s.groupby(by=['first', 'second']).sum()

,,0
first,second,
bar,doo,-0.012138
baz,bee,-1.394844
foo,bop,2.237990
qux,bop,1.244382


## Grouping DataFrame with Index levels and columns